In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import math

train_raw = pd.read_csv("Data_Train_.csv")

In [2]:
train_raw.head()

,Name,Brand1,Brand2,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Mileage_kmpl,Engine,Power,Seats,New_Price,Price
0,Maruti Wagon R LXI CNG,Maruti,Wagon,mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,NaN,998.0,58.16,5.0,NaN,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Hyundai,Creta,pune,2015,41000,Diesel,Manual,First,19.67 kmpl,NaN,1582.0,126.20,5.0,NaN,12.50
2,Honda Jazz V,Honda,Jazz,chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,NaN,1199.0,88.70,5.0,8.61,4.50
3,Maruti Ertiga VDI,Maruti,Ertiga,chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,NaN,1248.0,88.76,7.0,NaN,6.00
4,Audi A4 New 2.0 TDI Multitronic,Audi,A4,coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,NaN,1968.0,140.80,5.0,NaN,17.74


In [3]:
Mileage_new = []
for i in train_raw.Mileage:
    if'km/kg' in i:
        Mileage_new.append(float(i.replace('km/kg',""))/1.3564471935108)
    else:
        Mileage_new.append(float(i.replace('kmpl',""))/1.3564471935108)

In [4]:
train_raw.Mileage_kmpl = Mileage_new

In [5]:
for i in range(len(train_raw.New_Price)):
    if type(train_raw["New_Price"][i]) == str:
        if 'Cr' in train_raw["New_Price"][i]:
            train_raw["New_Price"][i] = float(train_raw["New_Price"][i].replace('Cr',''))
        else:
            train_raw["New_Price"][i] = float(train_raw["New_Price"][i])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [6]:
train_raw.isna().sum()

Name                    0
Brand1                  0
Brand2                  0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 0
Mileage_kmpl            0
Engine                 36
Power                 143
Seats                  42
New_Price            5194
Price                   0
dtype: int64

In [7]:
train_raw.shape

(6017, 16)

In [8]:
train_raw[train_raw.Seats.isna()]

,Name,Brand1,Brand2,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Mileage_kmpl,Engine,Power,Seats,New_Price,Price
194,Honda City 1.5 GXI,Honda,City,ahmedabad,2007,60006,Petrol,Manual,First,0.0 kmpl,0.000000,NaN,NaN,NaN,NaN,2.95
208,Maruti Swift 1.3 VXi,Maruti,Swift,kolkata,2010,42001,Petrol,Manual,First,16.1 kmpl,11.869242,NaN,NaN,NaN,NaN,2.11
229,Ford Figo Diesel,Ford,Figo,bangalore,2015,70436,Diesel,Manual,First,0.0 kmpl,0.000000,1498.0,99.0,NaN,NaN,3.60
733,Maruti Swift 1.3 VXi,Maruti,Swift,chennai,2006,97800,Petrol,Manual,Third,16.1 kmpl,11.869242,NaN,NaN,NaN,NaN,1.75
749,Land Rover Range Rover 3.0 D,Land,Rover,mumbai,2008,55001,Diesel,Automatic,Second,0.0 kmpl,0.000000,NaN,NaN,NaN,NaN,26.50
1294,Honda City 1.3 DX,Honda,City,delhi,2009,55005,Petrol,Manual,First,12.8 kmpl,9.436416,NaN,NaN,NaN,NaN,3.20
1327,Maruti Swift 1.3 ZXI,Maruti,Swift,hyderabad,2015,50295,Petrol,Manual,First,16.1 kmpl,11.869242,NaN,NaN,NaN,NaN,5.80
1385,Honda City 1.5 GXI,Honda,City,pune,2004,115000,Petrol,Manual,Second,0.0 kmpl,0.000000,NaN,NaN,NaN,NaN,1.50
1460,Land Rover Range Rover Sport 2005 2012 Sport,Land,Rover,coimbatore,2008,69078,Petrol,Manual,First,0.0 kmpl,0.000000,NaN,NaN,NaN,NaN,40.88
1917,Honda City 1.5 EXI,Honda,City,jaipur,2005,88000,Petrol,Manual,Second,13.0 kmpl,9.583860,1493.0,100.0,NaN,NaN,1.70


In [9]:
train_raw.drop(["Name","New_Price","Mileage"],inplace=True,axis=1)

In [10]:
cat_columns = ['Brand1', 'Brand2', 'Location','Fuel_Type', 'Transmission', 'Owner_Type']
train_raw_dummies = pd.get_dummies(train_raw, prefix_sep="__",
                              columns=cat_columns, drop_first= True)

In [11]:
train_raw_dummies

,Year,Kilometers_Driven,Mileage_kmpl,Engine,Power,Seats,Price,Brand1__Audi,Brand1__BMW,Brand1__Bentley,...,Location__kolkata,Location__mumbai,Location__pune,Fuel_Type__Diesel,Fuel_Type__LPG,Fuel_Type__Petrol,Transmission__Manual,Owner_Type__Fourth & Above,Owner_Type__Second,Owner_Type__Third
0,2010,72000,19.610052,998.0,58.16,5.0,1.75,0,0,0,...,0,1,0,0,0,0,1,0,0,0
1,2015,41000,14.501117,1582.0,126.20,5.0,12.50,0,0,0,...,0,0,1,1,0,0,1,0,0,0
2,2011,46000,13.417404,1199.0,88.70,5.0,4.50,0,0,0,...,0,0,0,0,0,1,1,0,0,0
3,2012,87000,15.312059,1248.0,88.76,7.0,6.00,0,0,0,...,0,0,0,1,0,0,1,0,0,0
4,2013,40670,11.205744,1968.0,140.80,5.0,17.74,1,0,0,...,0,0,0,1,0,0,0,0,1,0
5,2012,75000,15.555342,814.0,55.20,5.0,2.35,0,0,0,...,0,0,0,0,1,0,1,0,0,0
6,2013,86999,17.015038,1461.0,63.10,5.0,3.50,0,0,0,...,0,0,0,1,0,0,1,0,0,0
7,2016,36000,8.374819,2755.0,171.50,8.0,17.50,0,0,0,...,0,1,0,1,0,0,0,0,0,0
8,2013,64430,15.142499,1598.0,103.60,5.0,5.20,0,0,0,...,0,0,1,1,0,0,1,0,0,0
9,2012,65932,16.440006,1248.0,74.00,5.0,1.95,0,0,0,...,0,0,0,1,0,0,1,0,1,0


In [12]:
train_raw_dummies_ = train_raw_dummies.copy()

In [13]:
def impute(df_, label):
    df = df_.copy()

    test_index= list(df[df[label].isna()].index)
    train_index = list(set(list(df.index)) - set(test_index))

    features = df.drop([label],axis=1)

    imputer = Imputer()
    transformed_values = imputer.fit_transform(features)
    features = pd.DataFrame(transformed_values, columns=features.columns)

    X_train = features.loc[train_index]
    Y_train = df.loc[train_index][label]

    X_test = features.loc[test_index]

    print(Y_train.isna().sum())
    rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
    rf.fit(X_train, Y_train)
    Y_output = rf.predict(X_test)

    for i,j in enumerate(test_index):
        df_[label][j] = Y_output[i]
        
    return df_

def results(y_test, prediction):
    print("R2 : "+str(r2_score(y_test, prediction))+"MAE : "+str(mean_absolute_error(y_test, prediction))+"RMSE : "+str(math.sqrt(mean_squared_error(y_test, prediction))))

In [14]:
for col in ["Engine", "Seats", "Power"]:
    train_raw_dummies = impute(train_raw_dummies, col)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


0


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


0


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


0


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [15]:
sum(train_raw_dummies.isna().sum())

0

In [22]:
X_train, X_test, y_train, y_test = train_test_split(train_raw_dummies.drop(["Price"],axis = 1), train_raw_dummies.Price, test_size=0.3, random_state=42)

In [23]:
rf = RandomForestRegressor(n_estimators = 500, random_state = 42)
rf.fit(X_train, y_train)
prediction = rf.predict(X_test)

In [24]:
results(y_test, prediction)

R2 : 0.9038992618412603MAE : 1.4338789031798762RMSE : 3.391154219084439


In [44]:
!pip install catboost

In [45]:
import catboost

In [46]:
cat_model = catboost.CatBoostRegressor()
cat_model.fit(X_train, y_train)

In [48]:
cat_predict = cat_model.predict(X_test)

In [49]:
results(y_test, cat_predict)

R2 : 0.9096170083912416MAE : 1.4635206997877295RMSE : 3.2887248116094066
